<a href="https://colab.research.google.com/github/kms040411/GTSRB_recognition/blob/main/Test_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
import os
import pandas as pd

from PIL import Image, ImageFilter, ImageEnhance, PpmImagePlugin
import random

from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Global Parameters

In [2]:
batch_size = 100
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_path = "/content/drive/My Drive/CS470/Project/pretrained2.ckpt"
label_num = 43

# Transform Functions

In [3]:
def do_blur(img):
    blurred = img.filter(ImageFilter.BoxBlur(3))
    return blurred

def do_gaussianblur(img):
    blurred = img.filter(ImageFilter.GaussianBlur(3))
    return blurred

def do_decolorization(img):
    grayscaled = img.convert("LA")
    grayscaled = grayscaled.convert("RGB")
    return grayscaled

def do_darkening(img):
    enhancer = ImageEnhance.Brightness(img)
    darkened = enhancer.enhance(0.1)
    return darkened

def do_exposure(img):
    enhancer = ImageEnhance.Contrast(img)
    contrasted = enhancer.enhance(5.0)
    enhancer2 = ImageEnhance.Brightness(contrasted)
    exposured = enhancer.enhance(5.0)
    return exposured

def do_noise(img):
    noisy = img.effect_spread(3)
    return noisy

# Load Dataset from Kaggle

In [4]:
# We need to upload kaggle.json
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset from kaggle
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force
!unzip -q *.zip

 98% 597M/612M [00:12<00:00, 38.3MB/s]
100% 612M/612M [00:12<00:00, 50.9MB/s]
replace Meta.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [5]:
class GSTRB_test_dataset(torch.utils.data.Dataset):
  def __init__(self, root, csv):
    self.root = root
    self.csv = pd.read_csv(csv)
    self.basic_transform = data_transforms = transforms.Compose([
                                                              transforms.Resize((32, 32)),
                                                              transforms.ToTensor(),
                                                              transforms.Normalize((0.3337, 0.3064, 0.3171), (0.2672, 0.2564, 0.2629))
                                                            ])
    self.counter = 0
    self.transform_num = 1
    
  def __len__(self):
    ans = len(self.csv) * self.transform_num
    return ans

  def __getitem__(self, idx):
    image = Image.open(os.path.join(self.root, str(self.csv.iloc[idx // self.transform_num, 7])))
    label = torch.tensor(self.csv.iloc[idx // self.transform_num, 6]).long()
        
    # Apply transform
    transforms = [do_blur, do_gaussianblur, do_decolorization, do_darkening, do_exposure, do_noise, None]
    rand_transform = transforms[self.counter]
    rand_transform = None
    self.counter = self.counter + 1
    self.counter = self.counter % self.transform_num
    if rand_transform is not None:
      image = rand_transform(image)
    image = self.basic_transform(image)

    pack = (image, label)
    return pack

dataset_loader = GSTRB_test_dataset("/content/", "/content/Test.csv")
test_dataset = torch.utils.data.DataLoader(dataset_loader,
                                        batch_size=batch_size,
                                        shuffle=True,
                                        drop_last=True
                                        )

# Define Model

In [6]:
class SimpleCNN(nn.Module):
  def __init__(self):
    super(SimpleCNN, self).__init__()
    # Spatial Transformation Network
    self.localization = nn.Sequential(
                                        nn.Conv2d(3, 8, kernel_size=7),
                                        nn.MaxPool2d(2, stride=2),
                                        nn.ReLU(True),
                                        nn.Conv2d(8, 10, kernel_size=5),
                                        nn.MaxPool2d(2, stride=2),
                                        nn.ReLU(True)
                                    )
    
    # Regressor for the affine matrix
    self.fc_loc = nn.Sequential(
                                  nn.Linear(10 * 4 * 4, 32),
                                  nn.ReLU(True),
                                  nn.Linear(32, 3 * 2)
                              )

    # CNN
    self.conv1 = nn.Conv2d(in_channels=3, out_channels=100, kernel_size=5)
    self.relu1 = nn.ReLU()
    self.batchnorm1 = nn.BatchNorm2d(100)
    self.maxpool1 = nn.MaxPool2d(2)
    self.drop1 = nn.Dropout2d()
    self.conv2 = nn.Conv2d(in_channels=100, out_channels=150, kernel_size=3)
    self.relu2 = nn.ReLU()
    self.batchnorm2 = nn.BatchNorm2d(150)
    self.maxpool2 = nn.MaxPool2d(2)
    self.drop2 = nn.Dropout2d()
    self.conv3 = nn.Conv2d(in_channels=150, out_channels=250, kernel_size=3)
    self.relu3 = nn.ReLU()
    self.batchnorm3 = nn.BatchNorm2d(250)
    self.maxpool3 = nn.MaxPool2d(2)
    self.drop3 = nn.Dropout2d()
    

    # Full-Connected
    self.fc1 = nn.Linear(250 * 2 * 2, 350)
    self.relu4 = nn.ReLU()
    self.fc2 = nn.Linear(350, label_num)
    self.relu5 = nn.ReLU()

  def forward(self, x):
    output = x

    # Spatial Transformation Network
    output = self.localization(output)
    output = output.view(-1, 10 * 4 * 4)
    theta = self.fc_loc(output)
    theta = theta.view(-1, 2, 3)
    grid = torch.nn.functional.affine_grid(theta, x.size(), align_corners=False)
    output = torch.nn.functional.grid_sample(x, grid, align_corners=False)

    # CNN
    output = self.conv1(output)
    output = self.relu1(output)
    output = self.maxpool1(output)
    output = self.batchnorm1(output)
    output = self.drop1(output)

    output = self.conv2(output)
    output = self.relu2(output)
    output = self.maxpool2(output)
    output = self.batchnorm2(output)
    output = self.drop2(output)

    output = self.conv3(output)
    output = self.relu3(output)
    output = self.maxpool3(output)
    output = self.batchnorm3(output)
    output = self.drop3(output)

    # Full-Connected
    output = output.view(-1, 250 * 2 * 2)
    output = self.fc1(output)
    output = self.relu4(output)
    output = self.fc2(output)
    output = self.relu5(output)
    output = nn.functional.log_softmax(output, dim=1)
    return output

In [7]:
def weights_init(m):
  if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
    nn.init.normal_(m.weight, mean=0.5, std=1.0)
    nn.init.normal_(m.bias, mean=0.5, std=1.0)

model = SimpleCNN()
model = model.to(device)
model.load_state_dict(torch.load(model_path)['state_dict'])

<All keys matched successfully>

# Calculate Model Accuracy

In [8]:
def label(prediction):
  _, pred_label = torch.max(prediction, 1)
  return pred_label

In [9]:
model.eval()
total_answer, correct_answer = 0.0, 0.0
for batch_idx, (x, answer) in enumerate(test_dataset):
    with torch.no_grad():
      x, answer = x.cuda(), answer.cuda()
      prediction = model(x)
      total_answer += x.data.size(0)
      correct_answer += (label(prediction) == answer.data).sum().item()

print("Test Accuracy: " + str(100.0 * correct_answer/total_answer) + "%")

Test Accuracy: 99.15873015873017%
